<a href="https://colab.research.google.com/github/project-ccap/project-ccap.github.io/blob/master/2024notebooks/2024_0226ccap_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2021年5月版 word2vec デモ

- date: 2021_0510
- filename: 2021_0530ccap_word2vec.pynb
- author: 浅川伸一
- 概要:

In [1]:
import numpy as np
# 表示精度桁数の設定
np.set_printoptions(suppress=False, formatter={'float': '{:6.3f}'.format})

In [4]:
%config InlineBackend.figure_format = 'retina'
import torch

import math
import random
import numpy as np
import time
import gzip
import json
import sys
import re
import json

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

try:
    import jaconv
except ImportError:
    !pip install jaconv
    import jaconv


from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

if isColab:

    # termcolor を downgrade しないと colab ではテキストに色がつかない
    !pip install --upgrade termcolor==1.1
    import termcolor

    # 結果を保存するために Google Drive をマウントする
    import google.colab
    google.colab.drive.mount('/content/drive/')

    # GPU 情報を表示
    !nvidia-smi -L

    #!pip install ipynbname --upgrade > /dev/null

if isColab:
    # colab 上で MeCab を動作させるために，C コンパイラを起動して，MeCab の構築を行う
    # そのため時間がかかる。
    !apt install aptitude
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
    !pip install mecab-python3==0.7
    !pip install jaconv
    !pip install japanize_matplotlib

    import MeCab
    mecab_wakati = MeCab.Tagger('-Owakati').parse
    mecab_yomi = MeCab.Tagger('-Oyomi').parse

else:
    from ccap.mecab_settings import yomi as mecab_yomi
    from ccap.mecab_settings import wakati as mecab_wakati


# ここから下は，コード実行に関するバージョン情報などの情報源の取得と表示
from termcolor import colored

import platform
HOSTNAME = platform.node().split('.')[0]

import os
HOME = os.environ['HOME']

try:
    import ipynbname
except ImportError:
    !pip install ipynbname
    import ipynbname
FILEPATH = str(ipynbname.path()).replace(HOME+'/','')

import pwd
USER=pwd.getpwuid(os.geteuid())[0]

from datetime import date
TODAY=date.today()

import torch
TORCH_VERSION = torch.__version__

color = 'green'
print('日付:',colored(f'{TODAY}', color=color, attrs=['bold']))
print('HOSTNAME:',colored(f'{HOSTNAME}', color=color, attrs=['bold']))
print('ユーザ名:',colored(f'{USER}', color=color, attrs=['bold']))
print('HOME:',colored(f'{HOME}', color=color,attrs=['bold']))
print('ファイル名:',colored(f'{FILEPATH}', color=color, attrs=['bold']))
print('torch.__version__:',colored(f'{TORCH_VERSION}', color=color, attrs=['bold']))

  Preparing metadata (setup.py) ... done
  Created wheel for jaconv: filename=jaconv-0.3.4-py3-none-any.whl size=16415 sha256=b0bce3422b9fa2a11de3eb1d005ec97e61a18b69d70329ff0ce2ed235891e7c2
  Stored in directory: /root/.cache/pip/wheels/46/8f/2e/a730bf1fca05b33e532d5d91dabdf406c9b718ec85b01b1b54
Successfully built jaconv
  Preparing metadata (setup.py) ... done
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4832 sha256=b912e6b59588cfa7c42713d975810e89b7b659aa3c10c6ae94beaa6417bca340
  Stored in directory: /root/.cache/pip/wheels/a1/49/46/1b13a65d8da11238af9616b00fdde6d45b0f95d9291bac8452
Successfully built termcolor
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.4.0
    Uninstalling termcolor-2.4.0:
      Successfully uninstalled termcolor-2.4.0
Mounted at /content/drive/
/bin/bash: line 1: nvidia-smi: command not found
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The fol

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=bae40bfff7a5fbfadecd2c6f039f8b228822e8cfbbdba9d67a3820b32430d9cb
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.3 MB/s eta 0:00:00
日付: 2024-02-26
HOSTNAME: 4ebd54fb94e7
ユーザ名: root
HOME: /root
ファイル名: /fileId=https%3A%2F%2Fgithub.com%2Fproject-ccap%2Fproject-ccap.github.io%2Fblob%2Fmaster%2Fnotebooks%2F2021_0531ccap_word2vec.ipynb
torch.__version__: 2.1.0+cu121


In [2]:
%time
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール
# reference: https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n
!pip install mecab-python3

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs
Reading package lists...
Building dependency tree...
Reading state information...
E: Unable to locate package python-mecab
Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Receiving objects: 100% (75/75), 58.09 MiB | 35.13 MiB/s, done.
Resolving deltas: 100% (5/5), done.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab is not found.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 7.6 MB/s eta

In [3]:
# 動作確認
import MeCab
neologd_path = "-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd"
m = MeCab.Tagger(neologd_path +' -Oyomi')
print(m.parse('鬼滅の刃'))

RuntimeError: 
----------------------------------------------------------

Failed initializing MeCab. Please see the README for possible solutions:

    https://github.com/SamuraiT/mecab-python3#common-issues

If you are still having trouble, please file an issue here, and include the
ERROR DETAILS below:

    https://github.com/SamuraiT/mecab-python3/issues

issueを英語で書く必要はありません。

------------------- ERROR DETAILS ------------------------
arguments: -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Oyomi
default dictionary path: None
[ifs] no such file or directory: /usr/local/etc/mecabrc
----------------------------------------------------------


In [ ]:
#訓練済 word2vec ファイルの取得
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1B9HGhLZOja4Xku5c_d-kMhCXn1LBZgDb" -O 2021_05jawiki_hid128_win10_neg10_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1OWmFOVRC6amCxsomcRwdA6ILAA5s4y4M" -O 2021_05jawiki_hid128_win10_neg10_sgns.bin.gz && rm -rf /tmp/cookies.txt
!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JTkU5SUBU2GkURCYeHkAWYs_Zlbqob0s" -O 2021_05jawiki_hid200_win20_neg20_cbow.bin.gz && rm -rf /tmp/cookies.txt
#!wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VPL2Mr9JgWHik9HjRmcADoxXIdrQ3ds7" -O 2021_05jawiki_hid200_win20_neg20_sgns.bin.gz && rm -rf /tmp/cookies.txt

In [ ]:
%time
#直上のセルで取得したファイルの読み込み
# word2vec データ処理のため gensim を使う
import os
import sys
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

print('# word2vec データの読み込み')
print('# 訓練済 word2vec，訓練データは wikipedia 全文  読み込みに時間がかかります...', end="")
# ファイルの所在に応じて変更してください
w2v_base = '.'
#w2v_file='2021_05jawiki_hid128_win10_neg10_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid128_win10_neg10_sgns.bin.gz'
w2v_file='2021_05jawiki_hid200_win20_neg20_cbow.bin.gz'
#w2v_file='2021_05jawiki_hid200_win20_neg20_sgns.bin.gz'
asakawa_w2v_file = os.path.join(w2v_base, w2v_file)
asakawa_w2v = KeyedVectors.load_word2vec_format(asakawa_w2v_file,
                                                encoding='utf-8',
                                                unicode_errors='replace',
                                                binary=True)

w2v = asakawa_w2v
print('done')

In [ ]:
#以下は操作例：単語の後の数字はコサイン類似度を表している
word = '人工知能' #入力単語
topn = 20        #類似度が高い順に何語出力するか
word = input('単語を入力してください: ')
w2v.most_similar(word,topn=topn)

コサイン類似度とは，ベクトル間の類似度を角度 $\theta$ として表現したもの。

$$
\text{$\mathbf{x}$ と $\mathbf{y}$ とのコサイン類似度 $\theta$}=\frac{(\mathbf{x}\cdot\mathbf{y})}{|\mathbf{x}||\mathbf{y}|}
$$

ただし上式では $-1\le\theta\le1$ であるので，$[0,1]$ の範囲になるように，$\theta$ に 1 を加えて 0.5 倍してある。

In [ ]:
w2v.most_similar(positive=('王','女'), negative=('男'),topn=topn)

In [ ]:
w2v.most_similar(positive=('早稲田大学','関西'), negative=('関東'))

In [ ]:
w2v.most_similar(positive=('ジャイアンツ','アメリカ'), negative=('日本'))

In [ ]:
w2v.most_similar(positive=('鬼滅の刃'))

In [ ]:
!pip install jaconv       #単語をカタカナに変換するために使用するライブラリ
!pip install Levenshtein  #レーベンシュタイン距離を算出するライブラリ
import jaconv
import Levenshtein

In [ ]:
# word2vec の類似語上位 topn 語の読みを元にレーベンシュタイン距離を算出して表示
topn=10
target = input('word2vec で類似の単語をカタカナ表記してレーベンシュタイン距離を求めるデモです。単語を入力してください: ')
mecabtagger = MeCab.Tagger('-Oyomi -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
kata_target = mecabtagger.parse(target).strip()
word_list = [ent[0] for ent in w2v.most_similar(target,topn=topn)]
kata_word_list = [mecabtagger.parse(word).strip() for word in word_list]
kata_distances = [Levenshtein.distance(kata_target, kata_word) for kata_word in kata_word_list]
sorted_word_list = [word_list[i] for i in  np.argsort(kata_distances)] # [::-1]]

print(f'ターゲット語: {target} {kata_target}')
for i, word in enumerate(sorted_word_list):
    yomi = kata_word_list[word_list.index(sorted_word_list[i])]
    dist = kata_distances[word_list.index(sorted_word_list[i])]
    print(f'{i+1:2d}: {word} (読み:{yomi}) 距離:{dist}')

word2vec で類似の単語をカタカナ表記してレーベンシュタイン距離を求めるデモです。単語を入力してください: 鬼滅の刃
ターゲット語: 鬼滅の刃 キメツノヤイバ
 1: 紅蓮の弓矢 (読み:グレンノユミヤ) 距離:6
 2: 竈門炭治郎 (読み:カマドタンジロウ) 距離:8
 3: 吾峠呼世晴 (読み:ゴトウゲコヨハル) 距離:8
 4: 呪術廻戦 (読み:ジュジュツカイセン) 距離:8
 5: 進撃の巨人 (読み:シンゲキノキョジン) 距離:8
 6: 約束のネバーランド (読み:ヤクソクノネバーランド) 距離:10
 7: ゆらぎ荘の幽奈さん (読み:ユラギソウノユウナサン) 距離:10
 8: 炎炎ノ消防隊 (読み:エンエンノショウボウタイ) 距離:11
 9: 僕のヒーローアカデミア (読み:ボクノヒーローアカデミア) 距離:12
10: 転生したらスライムだった件 (読み:テンセイシタラスライムダッタケン) 距離:15
